In [1]:
# VISUALISESKELETON --- simulates the arm movement in time.
#                     Input:
#                       - filename: path of the mat structure of the action
#                      
#                     Output:
#                       Simulation of the arm executing the complete action 
#                       (multiple instances)
# 
# Example of use:
# filename = '../CookingDataset-master/data/mocap/training/carrot_tr.mat';
# visualizeSkeleton(filename);

# synchronizedView --- function that visualize an action in synch from video and from skeleton data
#                       Input:
#                           - vid: a VideoReader object
#                           - mocap: the stream of motion capture data
#                           - startMocap: starting instant for the MoCap
#                               stream
#                           - startVid: starting frame
#                           
# Example of use:
# mocap_folder = '../CookingDataset-master/data/mocap/training'
# video_folder = '.../CookingDataset-master/data/video/training'
# vid, mocap, startMocap, startVid,pov = intSynch('carrot', 'tr', 0, mocap_folder, video_folder)
# synchronizedView(vid, mocap, startMocap, startVid, pov)


import os
from os import path
import time
import sys
import csv
import pandas as pd
from scipy.io import loadmat 
import pylab
import cv2 

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import FileLink
from matplotlib.animation import FuncAnimation
from itertools import cycle
import matplotlib.cm as cm
# %matplotlib nbagg
%matplotlib qt

def intSynch(action, mode, pov, mocap_folder, video_folder):
    synch_indexfile="synch_index.csv"
    df_synchindex=pd.read_csv(synch_indexfile)

    df_row=df_synchindex.loc[df_synchindex['action'] == action].values.tolist()#[['carrot', 1500, 90, 601, 76]]
    
    if mode=='tr':
        startMocap = df_row[0][1]
        startVid =   df_row[0][2]
        mocap_file=path.join(mocap_folder,"training",action+"_"+mode+".mat")
        video_file=path.join(video_folder,"training",action.upper()+"_"+mode.upper()+"_"+str(pov)+".avi")
 
    elif mode == 'te':
        startMocap = df_row[0][3]
        startVid =   df_row[0][4]
        mocap_file=path.join(mocap_folder,"test",action+"_test.mat")
        video_file=path.join(video_folder,"test",action.upper()+"_"+mode.upper()+"_"+str(pov)+".avi")
    elif mode == 'scene':
        startMocap = df_row[0][1]
        startVid =   df_row[0][2]
        mocap_file=path.join(mocap_folder,"scenes",action.capitalize()+".mat")
        video_file=path.join(video_folder,"scenes",action.upper()+"_"+str(pov)+".avi")
    

    
    print("video_file:",video_file)
    vid = cv2.VideoCapture(video_file)
    print(vid)
    
    #motion capture
    column_names=['IND_X','IND_Y','IND_Z','PALM_X','PALM_Y','PALM_Z', 'LIT_X', 'LIT_Y','LIT_Z','WRIST_X','WRIST_Y','WRIST_Z', 'ELBOW_X','ELBOW_Y','ELBOW_Z', 'SHOULDER_X','SHOULDER_Y','SHOULDER_Z']
    df_mocap = pd.DataFrame(columns = column_names)

    single_action = loadmat(mocap_file)  # load mat-file in python

    arr_index=single_action['index'] # index array
    start_index=arr_index[0][0] # start index

    
    marker_list = ['IND','PALM', 'LIT', 'WRIST', 'ELBOW', 'SHOULDER']
    for marker in marker_list:
        arr = single_action[marker]

        x=marker+'_X'
        y=marker+'_Y'
        z=marker+'_Z'
        df_mocap[x]=arr[:,0]
        df_mocap[y]=arr[:,1]
        df_mocap[z]=arr[:,2]
    
    mocap=df_mocap
    return vid, mocap, startMocap, startVid, pov



def synchronizedView(vid, mocap, startMocap, startVid, pov):
    df= mocap
    
    if startVid == 0:
        vid_frame = 1;
    else:
        start = startVid;
        vid_frame = start;

    if startMocap == 0:
        mocap_frame=1;
    else:
        mocap_frame= startMocap
    
    #Adjust the point of view of the skeleton plot with the video 
#     if pov==0: 
#         az = -40;
#         el = 18;
#     if pov==1:
#         az = -180;
#         el = 90;
#     if pov==2:
#         az = 0;
#         el = 50;
    
    fig = plt.figure()
        
    while(True):

        success,image = vid.read()

        if success:  
            ax1 = fig.add_subplot(1, 2,1)
            ax1.imshow(image)
            ax1.set_title("Video Frame:"+ str(vid_frame))
            
            ax2 = fig.add_subplot(1, 2,2, projection='3d')
            #ax2.view_init(az,el)

            ax2.set_title("Mocap Frame:"+ str(mocap_frame))
            ax2.set_xlim([-500,600])
            ax2.set_ylim([-400,600])
            ax2.set_zlim([-100,500])
            ax2.set_xlabel('X', fontsize=10, rotation=150)
            ax2.set_ylabel('Y',fontsize=10)
            ax2.set_zlabel('Z', fontsize=10, rotation=60)
            
            i=mocap_frame
            ind_x= df['IND_X'][i]
            ind_y= df['IND_Y'][i]
            ind_z= df['IND_Z'][i]
            palm_x= df['PALM_X'][i]
            palm_y= df['PALM_Y'][i]
            palm_z= df['PALM_Z'][i]
            lit_x= df['LIT_X'][i]
            lit_y= df['LIT_Y'][i]
            lit_z= df['LIT_Z'][i]
            wri_x= df['WRIST_X'][i]
            wri_y= df['WRIST_Y'][i]
            wri_z= df['WRIST_Z'][i]
            elb_x= df['ELBOW_X'][i]
            elb_y= df['ELBOW_Y'][i]
            elb_z= df['ELBOW_Z'][i]
            sho_x= df['SHOULDER_X'][i]
            sho_y= df['SHOULDER_Y'][i]
            sho_z= df['SHOULDER_Z'][i]
    
            x1= [sho_x,elb_x]
            y1= [sho_y,elb_y]
            z1= [sho_z,elb_z]
        
            x2= [elb_x,wri_x]
            y2= [elb_y,wri_y]
            z2= [elb_z,wri_z]
        
            x3= [wri_x, palm_x]
            y3= [wri_y, palm_y]
            z3= [wri_z, palm_z]
        
            x4= [palm_x,ind_x]
            y4= [palm_y,ind_y]
            z4= [palm_z,ind_z]
        
            x5= [palm_x,lit_x]
            y5= [palm_y,lit_y]
            z5= [palm_z,lit_z]
        
            ax2.plot(x1, y1,z1 ,'k-' )
            ax2.plot(x2, y2,z2,'k-')
            ax2.plot(x3, y3,z3 ,'k-' )
            ax2.plot(x4, y4,z4,'k-')
            ax2.plot(x5, y5,z5,'k-')

            ax2.scatter(ind_x, ind_y,ind_z,c='b' ,label='IND')
            ax2.scatter(palm_x, palm_y,palm_z, c='green',label='PALM')
            ax2.scatter(lit_x, lit_y,lit_z, c='red',label='LIT')
            ax2.scatter(wri_x, wri_y,wri_z,  c='c',label='WRIST')
            ax2.scatter(elb_x, elb_y,elb_z,  c='y',label='ELBOW')
            ax2.scatter(sho_x, sho_y,sho_z,  c='m',label='SHOULDER')
            ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))

        


            plt.pause(0.5)
            ax1.remove()
            ax2.remove()
            
            #STEP: 3 video frames corresponds to 10 MoCap points in space
            vid_frame = vid_frame+15;
            mocap_frame = mocap_frame+50;
            vid.set(1, vid_frame)

            if mocap_frame > df.shape[0]:
                break
            
        else: 
            break
        
    plt.close() 
        

In [2]:
 # Function Calling   
mocap_folder = '../CookingDataset-master/data/mocap/';
video_folder = '../CookingDataset-master/data/video/';   
action="scene1" # for scenes 
mode="scene"
pov=0

vid, mocap, startMocap, startVid, pov=intSynch(action, mode, pov, mocap_folder, video_folder)
synchronizedView(vid, mocap, startMocap, startVid, pov)

video_file: ../CookingDataset-master/data/video/scenes/SCENE1_0.avi
<VideoCapture 0x7ff301c82f50>
